<a href="https://colab.research.google.com/github/leeCodingStudio/Python_NLP/blob/master/8_%ED%8A%B8%EB%9E%9C%EC%8A%A4%ED%8F%AC%EB%A8%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 트랜스포머(Transformer)
* 2017년 구글이 발표한 논문 "Attention is all you need"에서 발표된 모델
* seq2seq의 구조인 인코더-디코더를 따르면서도, 어텐션(Attention)만으로 구현된 모델
* RNN을 사용하지 않고, 인코더-디코더 구조를 설계하였음에도 번역 성능에서도 RNN보다 우수한 성능을 보여주었으며 2017년 이후 지금까지도 다양한 분야에서 사용되는 범용적인 모델

### 1-1. 트랜스포머의 특징
* 인코더가 입력 문장의 정보를 벡터로 압축할 때 입력 문장의 정보가 일부 손실됨을 보정하기 위해 어텐션 메커니즘이 제안되었으며 실제로 seq2seq 모델에 attention을 사용한 구조는 어느정도 성능 향상을 가져옴(여전히 RNN의 구조적인 문제로 한계점은 있음)
* 기존 RNN은 입력을 순차적으로 처리하기 때문에 병렬화가 불가능 했음 -> 어텐션 방식으로 생각해보면 시퀀스 간 관계를 파악할 때 병렬적으로 파악할 수 있음
* 트랜스포커는 RNN을 사용하지 않지만 seq2seq 모델의 구조처럼 인코더에서 입력 시퀀스를 입력받고, 디코더에서 출력 시퀀스를 출력하는 인코더-디코더를 사용함
* seq2seq 모델 구조에서는 인코더와 디코더 각각 하나의 RNN 모델처럼 작동했다면, 트랜스포머에서는 인코더와 디코더 단위가 N개로 확장되는 구조(논문에서는 6개씩 사용)
* 포지셔널 인코딩(positional encoding): 트랜스포머는 단어의 위치정보를 얻기 위해 각 단어의 임베딩 벡터에 위치 정보들을 더하여 모델의 입력으로 사용
* 위치 정보를 가진 값을 만들기 위해 sin, cos 함수를 사용(pos는 입력 문장에서의 임베딩 벡터의 위치를 나타냄, i는 임베딩 벡터 내의 차원의 인덱스를 의미)
* 임베딩 벡터 내의 차원의 인덱스가 짝수인 경우에는 사인 함수의 값을 사용하고, 홀수인 경우에는 코사인 함수의 값을 사용하는 구조(예: 임베딩 벡터 내위치가 (pos, 2i)일 때는 사인 함수를 사용하고, (pos, 2i+1)일 때는 코사인 함수를 사용)
* 각 임베딩 벡터에 포지셔널 인코딩의 값을 더하면 같은 단어라고 하더라도 문장내의 위치에 따라 트랜스포머의 입력으로 들어가는 임베딩 벡터의 값이 달라짐
* https://wikidocs.net/31379

### 1-3. 트랜스포머의 셀프 어텐션
* 어텐션을 스스로에게 수행한다는 의미
* 기존의  seq2seq 모델에서 사용하는 어텐션은 디코더가 생성하는 특정 시점의 hidden state와 입력 문장들과의 관계를 계산하기 위해 사용. 이때 Query는 디코더가 생성하는 특정시점의 Hidden state이며, key와 value는 입력 문장 내 단어 각각의 hidden state를 사용함.
* 어텐션 함수는 주어진 '쿼리'에 대해서 모든 '키'와의 유사도를 가중치로 하여 키와 맵핑 되어 있는 '값'에 반영한 후 유사도가 반영된 '값'을 모두 종합하여 어텐션 분포를 통해 입력 문장의 단어별 가중치를 계산
    :두꺼운_확인_표시:쿼리 : key와의 유사도
* 셀프 어텐션은 하나의 문장 내에서 단어간의 관계를 파악하기 위해 사용하는 어텐션 메커니즘으로 Query, key, value가 동일
* Q,K,V 백터를 얻었다면, 기존의 어텐션 메커니즘과 동일하게 작동한다.
    * 첫번째 : 각 Q백터는 모든 K백터에 대해 어텐션 스코어를 구한다.
    * 두번쨰 : 각 Q백터는 모든 K백터에 대해 어텐션 분포를 구한다.
    * 세번째 : 어텐션 스코어와 어텐션 분포를 V백터에 가중합하여 어텐션 값 or 컨텍스트 백터를 구한다.
    * 네번째 :Q백터에 대해 반복
* **각 단어에 대한 Q,K,V 백터를 구하고 :큰_노란색_원:스케일드 닷 프로덕트 어텐션:큰_노란색_원:으로 수행했던 과정들을 백터 연산이 아니라 행렬 연산을 하게 되면 일괄 계산이 가능**
<center><img alt="" src="https://wikidocs.net/images/page/31379/transformer10.png"></center>

* 그 동물은 길을 건너지 않았다, 왜냐면 그것은 피곤했기 때문이다.
* 여기서 그것이 동물인지 길인지 파악하는 것

### 1-4. 멀티헤드 어텐션
* 어텐션에서는 d_model의 차원을 가진 단어 벡터를 num_heads로 나눈 차원으로 어텐션을 수행
* 논문 기준, 512의 차원의 각 단어 벡터를 8로 나눠 64차원의 Q, K, V 벡터로 바꿔서 어텐션을 수행한 셈
* 트랜스포머 연구진은 한 번의 어텐션을 하는 것보다 여러 번의 어텐션을 병렬로 사용하는 것이 더 효과적이라고 판단
* d_model의 차원을 num_head개로 나누면 d_model/num_head의 차원을 가지는 Q, K, V에 대해서 어텐션을 num_head번 수행할 수 있음
* 어텐션 메커니즘이 병렬로 이뤄지며 각각의 어텐션 값 행렬을 어텐션 헤드라 함
* d_model의 단어 임베딩을 차원 축소하는데 사용되는 가중치 행렬의 값이 8개의 어텐션 헤드마다 전부 다름
* 병렬 어텐션을 모두 수행하면 모든 어텐션 헤드를 연결하며, 모두 연결된 어텐션 헤드 행렬의 크기는 (seq_len, d_model)

### 1-5. Position-wise FFNN(Feed Forward Neural Network)
* 우리가 알고있는 일반적인 deep neural nerwork의 feed forward 신경망
* 각각의 학습 노드가 서로 완전하게 연결된 Fully-connected NN이라고 해석할 수 있음

### 1-6. 잔차 연결(Residual Connection)
* sublayer의 출력에 입력 값을 더해주는 기법
* 입력과 출력은 FFNN을 지나기 때문에 동일한 차원을 가지므로 덧셈이 가능

### 1-7. 레이어 정규화(Layer Normalization)
* Residual Connection을 거친 결과는 이어서 Layer Normalization 과정을 거침
* 수식으로 구현된 인코더는 총 num_layers만큼을 순차적으로 한 후에 마지막 층의 인코더의 출력을 디코더에게 전달하면서 디코더 연산이 시작
* 디코더 또한 num_layer만큼의 연산을 하는데, 이때마다 인코더가 보낸 출력을 디코더 layer 각각에서 모두 사용

### 1-8. 트렌스포머의 디코더(Decoder)
* 디코더도 인코더와 동일하게 임베딩 층과 포지셔널 인코딩을 거친 후에 문장 행렬이 입력
* Teacher Forcing 기법을 사용하여 훈련되므로 학습 과정에서 디코더는 정답 문장에 해당하는 문장 행렬을 한 번에 입력
* seq2seq는 정답을 하나씩 넣어주지만, 트랜스포머는 문장을 통째로 넣어주기 때문에, 현 시점의 정답이 아니라 이후에 나올 정답 단어들까지 참조

### 1-9. Look-ahead maks
* 룩-어헤드 마스크는 디코더의 첫번째 sublayer애서 이루어짐
* 디코더의 첫번째 sublayer인 헤드 셀프 어텐션에서 어텐션 스코어 행렬에 마스킹을 적용

# 2. 트랜스포머 구현
* 위에서 알아본 인코더와 디코더 구조를 조합하여 트랜스포머를 만들 수 있음
* 인코더의 출력은 디코더에서 인코더-디코더 어텐션에서 사용하기 위해 디코더로 전달되며 디코더의 마지막에는 다중 클래스 분류 문제를 풀수 있도록 vocab_size 만큼의 크기를 가지는 Dense Lauer가 있음